In [1]:
import sys
sys.path.append("../")

import pandas as pd
import plotly.graph_objects as go

In [2]:
from infrastructure.instrument_collection import instrumentCollection

In [3]:
def load_results(ic):
    res = []
    trades = []
    our_curr = ['USD', 'GBP', 'JPY', 'NZD']
    for p1 in our_curr:
        for p2 in our_curr:
            pair = f"{p1}_{p2}"
            if pair in ic.instruments_dict.keys():
                res.append(pd.read_pickle(f"./macd_ema/macd_ema_res_{pair}.pkl"))
                trades.append(pd.read_pickle(f"./macd_ema/trades/macd_ema_trades_{pair}.pkl"))
    return pd.concat(res), pd.concat(trades)

In [4]:
instrumentCollection.LoadInstruments('../data/')

In [5]:
df_results, df_trades = load_results(instrumentCollection)
df_results.reset_index(drop=True, inplace=True)
df_trades.reset_index(drop=True, inplace=True)

In [6]:
df_results.head()

,pair,slow,fast,ema,result,signal
0,USD_JPY,26,12,50,3.0,9
1,USD_JPY,26,12,100,0.0,9
2,USD_JPY,26,12,50,7.0,12
3,USD_JPY,26,12,100,8.5,12
4,USD_JPY,26,18,50,5.5,9


In [7]:
df_trades.head()

,running,start_index_m5,profit_factor,loss_factor,start_price,trigger_price,SIGNAL,TP,SL,result,end_time,start_time,slow,fast,signal,ema,pair
0,False,4065,1.5,-1.0,105.302,105.144,-1,105.1705,105.373,1.5,2020-10-21 07:10:00+00:00,2020-10-21 05:00:00+00:00,26,12,9,50,USD_JPY
1,False,5344,1.5,-1.0,104.424,104.032,-1,104.0485,104.656,1.5,2020-10-29 10:35:00+00:00,2020-10-27 17:00:00+00:00,26,12,9,50,USD_JPY
2,False,7068,1.5,-1.0,104.580,104.580,-1,104.7080,104.468,1.5,2020-11-04 18:00:00+00:00,2020-11-04 18:00:00+00:00,26,12,9,50,USD_JPY
3,False,10278,1.5,-1.0,103.799,103.853,-1,103.5160,103.821,-1.0,2020-11-19 22:00:00+00:00,2020-11-19 22:00:00+00:00,26,12,9,50,USD_JPY
4,False,13675,1.5,-1.0,103.993,104.049,-1,103.9035,104.031,-1.0,2020-12-07 18:20:00+00:00,2020-12-07 18:00:00+00:00,26,12,9,50,USD_JPY


In [8]:
df_results.columns

Index(['pair', 'slow', 'fast', 'ema', 'result', 'signal'], dtype='object')

In [9]:
# look at the results grouped by setting
gr = df_results.groupby(by=['slow', 'fast', 'ema', 'signal'], as_index=False).sum()

In [10]:
# sort descending, best result at the top
gr.sort_values(by='result', ascending=False, inplace=True)

In [11]:
gr.head()

,slow,fast,ema,signal,pair,result
14,52,18,100,9,USD_JPYGBP_USDGBP_JPYGBP_NZDNZD_USDNZD_JPY,19.0
11,52,12,100,12,USD_JPYGBP_USDGBP_JPYGBP_NZDNZD_USDNZD_JPY,16.5
12,52,18,50,9,USD_JPYGBP_USDGBP_JPYGBP_NZDNZD_USDNZD_JPY,15.5
9,52,12,50,12,USD_JPYGBP_USDGBP_JPYGBP_NZDNZD_USDNZD_JPY,14.0
13,52,18,50,12,USD_JPYGBP_USDGBP_JPYGBP_NZDNZD_USDNZD_JPY,14.0


In [12]:
# for the top setting, let's see what we have in terms of individual currency pairs
df_results_best = df_results[(df_results.ema==50)&(df_results.fast==18)&(df_results.slow==52)&(df_results.signal==9)]

In [13]:
df_results_best

,pair,slow,fast,ema,result,signal
12,USD_JPY,52,18,50,3.0,9
28,GBP_USD,52,18,50,-1.5,9
44,GBP_JPY,52,18,50,3.0,9
60,GBP_NZD,52,18,50,6.5,9
76,NZD_USD,52,18,50,0.5,9
92,NZD_JPY,52,18,50,4.0,9


In [14]:
# let's see the individual trades for the top setting
df_trades_best = df_trades[(df_trades.ema==50)&(df_trades.fast==18)&(df_trades.slow==52)&(df_trades.signal==9)].copy()

In [15]:
df_trades_best.columns

Index(['running', 'start_index_m5', 'profit_factor', 'loss_factor',
       'start_price', 'trigger_price', 'SIGNAL', 'TP', 'SL', 'result',
       'end_time', 'start_time', 'slow', 'fast', 'signal', 'ema', 'pair'],
      dtype='object')

In [16]:
# let's see how long trades last
df_trades_best['duration_h'] = (df_trades_best.end_time - df_trades_best.start_time).dt.total_seconds() / 3600

In [17]:
df_trades_best['duration_h'].describe()

count    52.000000
mean      8.171474
std      14.715581
min       0.000000
25%       0.000000
50%       1.791667
75%      11.041667
max      88.500000
Name: duration_h, dtype: float64

In [18]:
# win percentage
wp = (df_trades_best[df_trades_best.result>0].shape[0] / df_trades_best.shape[0]) * 100
print("Win percentage", wp)

Win percentage 51.92307692307693


In [19]:
# we will take a look at the cumulative gains, so sort by end_time
df_trades_best.sort_values(by='end_time', inplace=True)

In [20]:
df_trades_best['CUM_RES'] = df_trades_best.result.cumsum()

In [21]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_trades_best.end_time,
    y=df_trades_best['CUM_RES'],
    line=dict(width=2)
))

fig.update_yaxes(
    gridcolor="#1f292f"
)
fig.update_xaxes(
    gridcolor="#1f292f"
)

fig.update_layout(
    margin=dict(l=10,r=10,b=10,t=10),
    paper_bgcolor="#2c303c",
    plot_bgcolor="#2c303c",
    font=dict(size=8, color="#e1e1e1"),
    height=400
)

fig.show()

c:\Users\hugma\trading\Lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

